# Analyzing NASA Planetary Exploration Budgets in SQL


For much of the last 60 years, NASA has been at the forefront of exploring our solar system. In this live training, we'll see how much money they spent to do this.

For expensive science projects, not least those funded with public money, the price of conducting research is a huge consideration, so budgeting is important.

Here, we'll use a cleaned up version of the [public dataset](https://www.planetary.org/space-policy/planetary-exploration-budget-dataset) provided by the The Planetary Society.

## What is the total cost of all planetary missions over all time?

A good first step in any budgetary analysis is to determine how much money has been spent in total.

For this, we need the `mission_budgets` table. Each row represents the cost in a fiscal year, of one aspect of a project for one mission. There are 5 columns:

- `mission`: The name of the mission.
- `fiscal_year`: The year, for accounting purposes.
- `cost_type`: Fine-grained aspect of the project, e.g., "Spacecraft".
- `cost_group`: Broader aspect of the project, e.g., "Development/Implementation".
- `cost_MUSD`: Cost in million US dollars.


In [1]:
-- Calculate the total cost of all missions over all time
SELECT SUM("cost_MUSD") AS total_cost_MUSD
	FROM mission_budgets

,total_cost_musd
0,41406.554506


## What happens when you adjust for inflation?

Due to inflation, costs in the 1960s are not directly equivalent to those in the 2020s. We need to adjust for inflation in order to get a cost in current currency.

Correction factors are available in the `inflation` table. Each row represents an inflation adjustment for a time period relative to today. ther are two columns.

- `fiscal_year`: The year, for accounting purposes. Note that in 1976, inflation was especially high, so two values are provided. "1976" represents the start of the year, and "1976TQ" represents the third quarter onwards.
- `inflation_adjustment`: Multiply currency values from the past time by this number to get current currency values.

In [2]:
-- Calculate the total cost of all missions over all time, adjusted for inflation
SELECT SUM("cost_MUSD"*inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    	--ON mission_bugets.fiscal_year = inflation.fiscal_year #Longer way 

,adjusted_total_cost_musd
0,80992.593983


## Which was the most expensive mission?

The biggest, grandest missions make headlines, but at some point, someone always asks questions about how much things cost, and the biggest budgets are the first place people look for cost savings. Knowing which is the most expensive project is an essential task for anyone responsible for a budget.

In [3]:
-- Get the mission with the highest total cost
SELECT mission, SUM("cost_MUSD"*inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    	--ON mission_bugets.fiscal_year = inflation.fiscal_year #Longer way 
    GROUP BY mission
    ORDER BY adjusted_total_cost_MUSD DESC
    LIMIT 1;

,mission,adjusted_total_cost_musd
0,Viking,7208.541041


## How much was spent each year?

Understanding how budgets change over time is also important. NASA has not been immune to the wax and wane of the US economy over the decades, nor the change in political enthusiasm for planetary exploration. More generally, to understand your current budget, it's usually helpful to know the context of how much budget you or your predecessors had in previous years.

In [4]:
-- Calculate the inflation adjusted total cost per year
SELECT fiscal_year, SUM("cost_MUSD"*inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    	--ON mission_bugets.fiscal_year = inflation.fiscal_year #Longer way 
    GROUP BY fiscal_year
    ORDER BY fiscal_year

,fiscal_year,adjusted_total_cost_musd
0,1960,261.139746
1,1961,674.849760
2,1962,1624.488450
3,1963,2268.368675
4,1964,2412.935820
...,...,...
72,2031,149.605633
73,2032,149.590728
74,2033,149.593212
75,2034,128.050000


In [5]:
# Draw a bar plot of total cost vs. year
import plotly.express as px

px.bar(
    total_cost_by_year, 
    x = 'fiscal_year',
    y = 'adjusted_total_cost_musd'
    
)

From the table it can be observed that at the beggining of the year 1960 the total cost increased dramatically due to the space race between to EUA and URSS, then in 1970 with the Neil Amostrong moon landing the people interes to see more of the space increased, as in the mission Viking but then decrease at a high rate caused by a reccesion all the goverment found were placed to cover the market and the unemployment, but as the years pass we have seen an exponential increase in the space budget even for coming years. 

## How much was spent on each destination?

The 1960s was famous for the "Space Race" between the United States and Russia to be the first to get a man on the moon. In the time since then, attention has also been turned to the other planets in the solar system and to small bodies like asteroids and comets.

To analyze spending by destination, we'll need to look at the `mission_details` table. Each row represenats one mission. There are 4 columns:

- `mission`: The name of the mission.
- `mission_full_name`: The formal name of the mission.
- `destination`: The place in the solar system that the mission is exploring.
- `program`: The name of the NASA program that the mission is part of.

In [6]:
-- Calculate the inflation adjusted total cost per destination
SELECT destination, SUM("cost_MUSD"*inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    LEFT JOIN mission_details
    	USING(mission)
    	--ON mission_bugets.fiscal_year = inflation.fiscal_year #Longer way 
    GROUP BY destination
    ORDER BY adjusted_total_cost_MUSD DESC	 	

,destination,adjusted_total_cost_musd
0,Mars,31725.348594
1,Outer Planets,21827.564786
2,The Moon,12982.887263
3,Small Bodies,7365.860162
4,Venus,4796.703011
5,Mercury,1519.605957
6,Earth-Sun L1,427.220100
7,The Sun,347.404110


## How did spending by destination change over time?

In [18]:
-- Calculate the inflation adjusted total cost per year per destination
SELECT fiscal_year, destination, SUM("cost_MUSD"*inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    LEFT JOIN mission_details
    	USING(mission)
    	--ON mission_bugets.fiscal_year = inflation.fiscal_year #Longer way 
    GROUP BY fiscal_year, destination
    ORDER BY fiscal_year DESC, destination

,fiscal_year,destination,adjusted_total_cost_musd
0,2035,Outer Planets,128.050000
1,2034,Outer Planets,128.050000
2,2033,Outer Planets,128.050000
3,2033,Small Bodies,21.543212
4,2032,Outer Planets,128.050000
...,...,...,...
277,1963,Venus,150.506848
278,1962,The Moon,1248.174750
279,1962,Venus,376.313700
280,1961,The Moon,674.849760


In [19]:
# Draw a bar plot of total cost vs. year, colored by destination
px.bar (
    total_cost_by_date_and_destination,
    x = 'fiscal_year',
    y = 'adjusted_total_cost_musd',
    color = 'destination'
)

So as the fiscal year advanced the humans started to search the more closer destinations as the moon or mars, but as the technology and our undestanding of the outter space increased we reached outer planets. But since 90´s the budget for Mars missions has increased over other destinations, even the outer planets and small bodies.

## Which part of the mission is most expensive, on average?

In [9]:
SELECT cost_type, cost_group, AVG("cost_MUSD"*inflation_adjustment) AS avg_adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    	--ON mission_bugets.fiscal_year = inflation.fiscal_year #Longer way 
    GROUP BY cost_type, cost_group
    ORDER BY avg_adjusted_total_cost_MUSD DESC

,cost_type,cost_group,avg_adjusted_total_cost_musd
0,Implementation (incl LV),Development/Implementation (incl LV),207.098456
1,Spacecraft,Development/Implementation,187.287245
2,Development (incl LV),Development/Implementation (incl LV),154.433170
3,Science,Science & Experiments,118.827580
4,Formulation,Formulation,111.024862
5,Launch Vehicle(s),Launch Vehicle(s) & Support,95.519585
6,Development,Development/Implementation,76.286351
7,Implementation,Development/Implementation,67.326700
8,Experiments,Science & Experiments,49.610943
9,Launch Support,Launch Vehicle(s) & Support,39.911608


In [10]:
# Draw a bar plot of total cost vs. year, colored by destination
px.bar (
    mission_cost_MUSD,
    x = 'cost_type',
    y = 'avg_adjusted_total_cost_musd',
    color = 'cost_group'
)

## Which programs have more than one destination for their missions?

In [21]:
SELECT program, destination, COUNT(destination) AS Num_missions
FROM mission_details
GROUP BY program, destination
ORDER BY Num_missions DESC


,program,destination,num_missions
0,Mars Exploration,Mars,9
1,Discovery,Small Bodies,7
2,Mariner,Mars,3
3,,Mars,2
4,Pioneer,Outer Planets,2
5,Discovery,The Moon,2
6,New Frontiers,Small Bodies,2
7,Mariner,Venus,2
8,Discovery,Mars,2
9,,Outer Planets,2


In [22]:
# Draw a bar plot of total cost vs. year, colored by destination
px.bar (
    destionation_count,
    x = 'program',
    y = 'num_missions',
    color = 'destination'
)